# Mapping and SNP calling for all individuals

## Using a loop for mapping, with a single folder per sample

In [ ]:
for i in {1..20}
    do
        cd /ifb/data/mydatalocal/MappingAndSNP
        echo Clone$i;
        mkdir dirClone$i; 
        cd dirClone$i; 
        echo "Cloning Clone$i data";
        ln -s /ifb/data/mydatalocal/shortReads/Clone${i}_pairedFq1.fq /ifb/data/mydatalocal/MappingAndSNP/dirClone$i/Clone${i}_1.fastq;
        ln -s /ifb/data/mydatalocal/shortReads/Clone${i}_pairedFq2.fq /ifb/data/mydatalocal/MappingAndSNP/dirClone$i/Clone${i}_2.fastq;
        echo -e "\nMapping Clone$i\n";
        bwa mem -t 4 -R "@RG\tID:Clone${i}\tSM:Clone${i}" ../referenceCorrect.fasta Clone${i}_1.fastq Clone${i}_2.fastq > Clone$i.sam;
        samtools view -@4 -b -f 0x02 Clone$i.sam | samtools sort -@4 -o Clone$i.SORTED.bam;
        samtools index Clone$i.SORTED.bam;
        echo -e "\nCalling Clone$i";
        gatk MarkDuplicates -I Clone$i.SORTED.bam -M duplicates.$i.metrics -O Clone$i.SORTED.MD.bam;
        samtools index Clone$i.SORTED.MD.bam;
        gatk --java-options "-Xmx4g" HaplotypeCaller --native-pair-hmm-threads 4 -I Clone$i.SORTED.MD.bam -O Clone$i.g.vcf -R ../referenceCorrect.fasta -ERC GVCF;
    done

## Calling all samples on one raw VCF

In [ ]:
cd /ifb/data/mydatalocal/MappingAndSNP
#Loop to inflate the --variant option
OPTION=""
for i in {1..20}
do
    OPTION="${OPTION} --variant dirClone${i}/Clone${i}.g.vcf"
done
echo $OPTION
gatk CombineGVCFs -R referenceCorrect.fasta $OPTION -O rawSNP.vcf

# Have a Look to it

In [ ]:
head -n 1000 rawSNP.vcf | tail

## Compute the Genotypes *:D I know I am a bad person...*

In [ ]:
gatk --java-options "-Xmx4g" GenotypeGVCFs -R referenceCorrect.fasta -V rawSNP.vcf -O output.vcf

## Compute the SNP density along the chromosomes

In [ ]:
echo -e "Reference\t1000000\n" > /ifb/data/mydatalocal/MappingAndSNP/genome.txt
bedtools genomecov -bga -split -i /ifb/data/mydatalocal/MappingAndSNP/output.vcf -g /ifb/data/mydatalocal/MappingAndSNP/genome.txt > /ifb/data/mydatalocal/MappingAndSNP/density.csv

In [ ]:
head /ifb/data/mydatalocal/MappingAndSNP/density.csv